In [1]:
!nvidia-smi

Sun Oct 20 17:55:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
! pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=67853d871e9b78be91c3abbfac4338dc50ba662947b5c8f994bc9df417f8cbf9
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [3]:
!lscpu |grep 'Model name'

Model name:                           Intel(R) Xeon(R) CPU @ 2.20GHz


In [4]:
import numpy as np
import pycuda.autoinit
from pycuda import driver, gpuarray
from pycuda.compiler import SourceModule
import time
from prettytable import PrettyTable

In [32]:
# GPU CUDA ядро
mod = SourceModule("""
        __global__ void mult_gpu(double* A, double* B, double* C){
                const int row =  threadIdx.y + blockIdx.y * blockDim.y;
                const int column = threadIdx.x + blockIdx.x * blockDim.x;
                const int N = 2000;
                for(int i = 0; i < N; i++){
                        C[column + row * N] += A[i + row * N] * B[column + i * N];
                }
        }
""")

# CPU перемножение через циклы
def mult_cpu(A, B):
    C, n  = np.zeros((N, N)), range(N)
    for i in n:
        for j in n:
            for k in n:
                C[i, j] += A[i, k] * B[k, j]
    return C

In [33]:
sizes = [200, 500, 1000, 2000]
N = sizes[3]

A = np.random.randn(N, N)
B = np.random.randn(N, N)
C_gpu = np.ones((N, N))


In [34]:
# GPU перемножение

# создание блока и сетки для функции CUDA
block_size = [2, 2]
block, grid = (block_size[0], block_size[1], 1), (int((N + block_size[0] - 1) / 2), int((N + block_size[1] - 1) / 2))
mult = mod.get_function("mult_gpu")

# запуск функции и засекание времени
start_gpu = time.time()
mult(driver.In(A), driver.In(B), driver.Out(C_gpu), block = block, grid = grid)
driver.Context.synchronize()
time_gpu = time.time() - start_gpu

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


In [35]:
# CPU перемножение
start_cpu = time.time()
C_cpu = mult_cpu(A, B)
time_cpu = time.time() - start_cpu

In [36]:
# Результаты
print(f'GPU время выполнения: {time_gpu}')
print(f'CPU время выполнения: {time_cpu}')
print(f'Во сколько раз быстрее на GPU: time_cpu/time_gpu = {time_cpu/time_gpu}')

# Матрица С_np посчитана с помощью numpy.dot для проверки корректности результатов
C_np = np.dot(A,B)
if np.allclose(C_cpu, C_np) and np.allclose(C_gpu, C_np):
  print('Функции перемножения на CPU и GPU посчитали корректно')
else:
  print('Результаты не сходятся с проверкой numpy.dot()')

#print(f"C_gpu: {C_gpu} \n C_cpu: {C_cpu} \n C_np: {C_np}")

GPU время выполнения: 1.4184284210205078
CPU время выполнения: 5014.554000854492
Во сколько раз быстрее на GPU: time_cpu/time_gpu = 3535.288722744784
Функции перемножения на CPU и GPU посчитали корректно


In [14]:
CPU_times = np.zeros(5)
GPU_times = np.zeros(5)

In [38]:
# Сделаем массивы, содержащие время выполнения (при изменении индекса и N, результаты записываются по очереди)
CPU_times[3] = time_cpu
GPU_times[3] = time_gpu

array([0.00494742, 0.02683139, 0.22877789, 1.41842842, 0.        ])

In [42]:
#Сделаем красивую таблицу
table = PrettyTable(['Размер матриц', 'Время CPU', 'Время GPU', 'Ускорение'])
table.align = 'l'
table.add_row(['200x200', CPU_times[0], GPU_times[0], CPU_times[0]/GPU_times[0] ])
table.add_row(['500x500', CPU_times[1], GPU_times[1], CPU_times[1]/GPU_times[1] ])
table.add_row(['1000x1000', CPU_times[2], GPU_times[2], CPU_times[2]/GPU_times[2] ])
table.add_row(['2000x2000', CPU_times[3], GPU_times[3], CPU_times[3]/GPU_times[3] ])
print(table)

+---------------+-------------------+-----------------------+-------------------+
| Размер матриц | Время CPU         | Время GPU             | Ускорение         |
+---------------+-------------------+-----------------------+-------------------+
| 200x200       | 4.512691020965576 | 0.0049474239349365234 | 912.1294395450822 |
| 500x500       | 78.94668793678284 | 0.026831388473510742  | 2942.325851482597 |
| 1000x1000     | 636.0277841091156 | 0.22877788543701172   | 2780.110424109283 |
| 2000x2000     | 5014.554000854492 | 1.4184284210205078    | 3535.288722744784 |
+---------------+-------------------+-----------------------+-------------------+
